In [ ]:
#Deconvolution of disease subset

In [2]:
import os
os.chdir('/home/fsboedij/Data/MouseAtlas')
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)
import scanpy as sc
import copy
import torch
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import gdown
import sankey
import anndata

import pandas as pd

In [3]:
adata=sc.read("Data/Final_object/MLCA_final_object_28_05.h5ad")

In [7]:
adata.obs.geo.value_counts()

Unknown      483431
GSE228594     45237
GSE211356     44788
GSE200639     40134
GSE218247     27446
GSE129605     17858
GSE127465     15775
GSE171861     15713
GSE178517     15546
GSE124872     12516
GSE209738     10141
GSE133992      6311
Name: geo, dtype: int64

In [3]:
#Subsetting object to disease, inclusive of disease specific cell types
dis = adata[adata.obs['MLCA_or_query'] == 'Disease']

dis.obs['Decon'] = dis.obs['Level_2_5']

dis.obs['Decon'] = dis.obs['Decon'].astype(object)

dis.obs['Decon_viral'] = dis.obs['Decon']

dis.obs['Decon_viral'] = dis.obs['Decon_viral'].astype(object)

dis.obs['Decon_viral'].loc[dis.obs['Level_5'] == 'Viral induced monocytes'] = 'Viral induced monocytes'
dis.obs['Decon_viral'].loc[dis.obs['Level_5'] == 'Viral induced Gran'] = 'Viral induced Gran'
dis.obs['Decon_viral'].loc[dis.obs['Level_5'] == 'Viral induced fibroblast'] = 'Viral induced fibroblast'
dis.obs['Decon_viral'].loc[dis.obs['Level_5'] == 'Viral induced alveolar macrophages'] = 'Viral induced alveolar macrophages'
dis.obs['Decon_viral'].loc[dis.obs['Level_5'] == 'Viral induced interstitial macrophages'] = 'Viral induced interstitial macrophages'
dis.obs['Decon_viral'].loc[dis.obs['Level_5'] == 'Smoke Induced Alveolar Macrophages'] = 'Smoke Induced Alveolar Macrophages'

dis.obs['Decon_viral'].loc[dis.obs['Decon_viral'].isin(['Cd4 T cells','Cd8 T cells'])] = 'T cell lineage'
dis.obs['Decon_viral'].loc[dis.obs['Decon_viral'].isin(['AT I epithelial','AT II epithelial'])] = 'AT epithelial'

print(dis.obs.Decon_viral.value_counts())

/tmp/ipykernel_4013045/1212333651.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dis.obs['Decon_viral'].loc[dis.obs['Level_5'] == 'Viral induced monocytes'] = 'Viral induced monocytes'
/tmp/ipykernel_4013045/1212333651.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dis.obs['Decon_viral'].loc[dis.obs['Level_5'] == 'Viral induced Gran'] = 'Viral induced Gran'
/tmp/ipykernel_4013045/1212333651.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-v

T cell lineage                            120835
B cells                                    78919
Capillary endothelial                      67124
Interstitial macrophages                   33089
AT epithelial                              32913
Granulocyte                                32111
NK cells                                   29962
Monocytes                                  28496
Alveolar macrophages                       27583
Viral induced monocytes                    26591
Fibroblast                                 22039
Dendritic cells                            20656
Viral induced interstitial macrophages     10651
Other cells                                 5971
Venous endothelial                          5489
Viral induced Gran                          5199
Secretory                                   4725
Viral induced alveolar macrophages          3251
Multiciliated_Deuterosomal                  3139
Viral induced fibroblast                    1243
Smoke Induced Alveol

In [5]:
adata

AnnData object with n_obs × n_vars = 734896 × 43519
    obs: 'nCount_covid', 'nFeature_covid', 'orig.ident', 'Group', 'res.0', 'res.0.1', 'res.0.2', 'res.0.3', 'res.0.4', 'res.0.5', 'res.0.6', 'res.0.7', 'res.0.8', 'res.0.9', 'res.1', 'seurat_clusters', 'infection', 'cell_type', 'Mice', 'cell_label_short_0.6', 'nCount_covidneg', 'nFeature_covidneg', 'infection_neg', 'new_cell_type', 'study', 'CellTypes', 'CellTypes_short', 'samples', 'Timepoint', 'doublet', 'identifier.1', 'name', 'grouping', 'batch', 'cells', 'cluster', 'condition', 'age', 'sex', 'strain', 'treatment', 'tissue', 'time', 'platform', 'Tumor.or.healthy', 'Biological.replicate', 'Library', 'Barcode', 'Library.prep.batch', 'Total.counts', 'Percent.counts.from.mitochondrial.genes', 'Most.likely.Immgen.cell.type', 'Major.cell.type', 'Minor.subset', 'x', 'y', 'Geno', 'ID_old', 'hash.ID', 'SampleIDs', 'new.Celltypes', 'day', 'barcode', 'geo', 'predictions', 'clusters_1', 'clusters_2', 'clusters_3', 'n_genes_by_counts', 'total_

Randomly create 8 repeats of the reference matrix to create the most accurate reference for cellular deconvolution.

In [7]:
#Set up a loop to create 8 random iterations of reference matrix from the mouse atlas
for i in range(1, 9):
    dis.X.data = dis.layers['raw'].data.copy()
    decon = dis  #decon only diseases
    counts = decon.layers['raw']
    decon.X.data = counts.data.copy()
    batches = np.unique(decon.obs['Decon_viral'])
    batches = batches[batches != 'Other cells']
    batches = batches.astype(str)
    cell_indices = []
    
    for batch in batches:
        idx = np.where(decon.obs['Decon_viral'] == batch)[0]
        np.random.shuffle(idx)
        cell_indices += idx[:200].tolist()
    
    adata_filter = decon[cell_indices, :]
    
    matrix = adata_filter.X.toarray()
    
    mat = pd.DataFrame(data=matrix, index=adata_filter.obs["Decon_viral"], columns=adata_filter.var_names).T
    
    mat.to_csv(f'CellDecon_ReferenceMatrices/Disease_ref_{i}.tsv', sep='\t', index_label='GeneSymbol')
    print("saved_ref_"+str(i))

saved_ref_1
saved_ref_2
saved_ref_3
saved_ref_4
saved_ref_5
saved_ref_6
saved_ref_7
saved_ref_8


Create pseudobulk object from single cell

In [ ]:
# initialize the object
#Needs to be the CPM norm counts for the pseudo
adata.X.data = adata.layers['norm'].data.copy()
adata.obs['Pseudo'] = adata.obs['sample']
adata.obs['Pseudo'] = adata.obs['Pseudo'].astype(object)
#removed due to large proportion of epi cells
adata.obs['Pseudo'].loc[adata.obs['study'] == 'Mouse_Post_Sendai'] = 'Removed'

from adpbulk import ADPBulk
adpb = ADPBulk(adata, "Pseudo")

# perform the pseudobulking
pseudo = adpb.fit_transform().T

pseudo = pseudo.rename(columns=lambda x: x.replace('Pseudo.',''))

pseudo = pseudo.drop('Removed',axis=1)
pseudo.to_csv('/data/kkovacs/Python/Pseudo_Mouse_Atlas.tsv',sep = '\t', index_label='GeneSymbol')  

In [ ]:
###Here is the post cibersort files, the cell impute fraction outputs, 
### as CSV files

# make a dataframe of the samples of mice against the Decon celltypes, so it
# matches with the output files
table = pd.crosstab(dis.obs['Pseudo'],dis.obs['Decon_viral'], normalize='index')

correlation=[]

for i in range(1, 9):
    # Read the data
    data = pd.read_csv(f'/shared/homes/155227/work/R/data/MouseAtlas/Disease_results/CIBERSORTx_Ref{i}_Disease_Results.csv', sep=',', index_col='Mixture')
    
    # Remove unneeded columns
    data = data.drop(['Correlation', 'RMSE', 'P-value'], axis=1)
    
    # Rename indices to remove 'sample.'
    data = data.rename(index=lambda x: x.replace('sample.', ''))
    
    # Ensure the tables have the same index order
    com_rows = table.index.intersection(data.index)
    table_reindexed = table.reindex(com_rows)
    data_reindexed = data.reindex(com_rows)
    
    # Calculate the Spearman correlation
    corr_matrix = table_reindexed.corrwith(data_reindexed, method='spearman')
    
    # Rename the correlation results
    corr_matrix.name = f'Attempt {i}'
    
    # Append the correlation results to the list
    correlation.append(corr_matrix)

#Concatenate the content of the correlation list object into one dataframe
corr_table = pd.concat(correlation,axis=1)
corr_table = corr_table.drop('Other cells',axis=0)


#Save the file and take it to Prism for best figures
corr_table.to_csv('/shared/homes/155227/work/R/data/MouseAtlas/CellDecon/disease/corr_table_final_disease.csv')


corr_table = pd.read_csv('Corr_table_v2.csv',sep=',',index_col='Decon')

The Pseudobulk data is then used as the mixture file in CIBERSORTx. 

In [1]:
# Loop through the range of files
for i in range(1, 9):
    # Read the data
    data = pd.read_csv(f'Disease_results/CIBERSORTx_Ref{i}_Disease_Results.csv', sep=',', index_col='Mixture')
    
    # Create the table of actual abundances
    table = pd.crosstab(dis.obs['Pseudo'], dis.obs['Decon_viral'], normalize='index')
    
    # Remove unneeded columns
    data = data.drop(['Correlation', 'RMSE', 'P-value'], axis=1)
    
    # Drop unwanted rows and columns from the table
    table.drop('Removed', axis=0, inplace=True)
    table = table.drop('Other cells', axis=1)
    table = table.add_prefix('Actual_')
    
    # Determine the layout for subplots
    num_rows = 2  # Adjust as needed
    num_cols = 2  # Adjust as needed
    num_plots = int(np.ceil(len(table.columns) / (num_rows * num_cols)))
    subplots_per_plot = num_rows * num_cols
    
    # Create scatter plots
    for plot_num in range(num_plots):
        start_col = plot_num * subplots_per_plot
        end_col = min((plot_num + 1) * subplots_per_plot, len(table.columns))
        
        # Create a grid of subplots
        fig, axes = plt.subplots(num_rows, num_cols, figsize=(12, 6))
        
        # Flatten the axes array if it's a 2D array
        axes = axes.flatten()
        
        # Loop through the columns and create scatter plots for the current plot
        for j, (col_table, col_data) in enumerate(zip(table.columns[start_col:end_col], data.columns[start_col:end_col])):
            # Select the current subplot
            ax = axes[j]
            
            # Concatenate the columns of interest
            combined_data = pd.concat([table[col_table], data[col_data]], axis=1)
            
            # Create the scatter plot
            combined_data.plot(x=col_table, y=col_data, kind='scatter', ax=ax, legend=False)
            
            # Calculate and plot the line of best fit
            line = np.polyfit(combined_data[col_table], combined_data[col_data], 1)
            ax.plot(combined_data[col_table], np.polyval(line, combined_data[col_table]), color='red', label='Line of Best Fit')
            
            # Set subplot title
            ax.set_title(f'{col_table} vs {col_data}')
        
        # Adjust layout for better spacing
        plt.tight_layout()
        
        # Save the current plot
        plt.savefig(f'scatter_plots_panel_{plot_num + 1}_Ref{i}.png')
        plt.clf()

NameError: name 'pd' is not defined